**《深度学习之 PyTorch 实战》**

讲师作者：[土豆老师](https://iphysresearch.github.io)

本讲介绍的卷积神经网络（convolutional neural network，CNN）是一类强大的神经网络，正是为处理图像数据而设计的。基于卷积神经网络结构的模型在计算机视觉领域中已经占主导地位，当今几乎所有的图像识别、对象检测或语义分割相关的学术竞赛、商业应用都以这种方法为基础。

现代卷积神经网络的设计得益于生物学、群论和大量的实验研究。除了在获得精确模型的采样效率外，卷积神经网络在计算上也是极其高效的。这是因为卷积神经网络需要的参数比多层感知机少，而且卷积神经网络很容易用GPU并行计算。因此，实践者经常尽可能多地应用卷积神经网络。即使在一维序列结构的任务上（例如音频、文本和时间序列分析），通常大家使用的是循环神经网络，而实践者也会经常使用到卷积神经网络。通过对卷积神经网络一些巧妙的调整，也使它们在图结构数据和推荐系统中发挥作用。

在本讲的开始，我们将介绍构成所有卷积网络主干的基本元素。这包括卷积层本身、填充（padding）和步幅（stride）、用于在相邻空间区域聚集信息的池化层（pooling）、每层中多通道（channel）的使用以及有关现代卷积网络架构的全面介绍。在本章的最后，我们将介绍一个完整的、可运行的 LeNet 模型：这是第一个卷积神经网络，早在现代深度学习兴起之前就已经得到成功应用。在下一讲中，我们将深入研究一些流行的、相对较新并具有一定代表性的卷积网络架构。

# 二维卷积层

卷积神经网络（convolutional neural network）是含有卷积层（convolutional layer）的神经网络。本章中介绍的卷积神经网络均使用最常见的二维卷积层。它有高和宽两个空间维度，常用来处理图像数据。本节中，我们将介绍简单形式的二维卷积层的工作原理。

## 互相关运算

虽然卷积层得名于卷积（convolution）运算，但我们通常在卷积层中使用更加直观的互相关（cross-correlation）运算。在二维卷积层中，一个二维输入数组和一个二维核（kernel）数组通过互相关运算输出一个二维数组。 我们用一个具体例子来解释二维互相关运算的含义。如下图所示，输入是一个高和宽均为 3 的二维数组。我们将该数组的形状记为 3×3 或（3，3）。核数组的高和宽分别为 2。该数组在卷积计算中又称卷积核或过滤器（filter）。卷积核窗口（又称卷积窗口）的形状取决于卷积核的高和宽，即 2×2。下图中的阴影部分为第一个输出元素及其计算所使用的输入和核数组元素：$0×0+1×1+3×2+4×3=19$。

![](https://i.loli.net/2021/04/14/jKgC8qvA7RBSasf.png)

在二维互相关运算中，卷积窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。当卷积窗口滑动到某一位置时，窗口中的输入子数组与核数组按元素相乘并求和，得到输出数组中相应位置的元素。图中的输出数组高和宽分别为 2，其中的 4 个元素由二维互相关运算得出：

$$
\begin{array}{l}
0 \times 0+1 \times 1+3 \times 2+4 \times 3=19 \\
1 \times 0+2 \times 1+4 \times 2+5 \times 3=25 \\
3 \times 0+4 \times 1+6 \times 2+7 \times 3=37 \\
4 \times 0+5 \times 1+7 \times 2+8 \times 3=43
\end{array}
$$

下面我们将上述过程实现在 `corr2d` 函数里。它接受输入数组 `X` 与核数组 `K`，并输出数组 `Y`。

In [1]:
import torch
from torch import nn

def corr2d(X, K):  # 本函数已保存在 dl4wm 包中方便以后使用
    """计算二维互相关运算。"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

我们可以构造图中的输入数组 `X`、核数组 `K` 来验证二维互相关运算的输出。

In [2]:
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[0, 1], [2, 3]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## 二维卷积层

二维卷积层将输入和卷积核做互相关运算，并加上一个标量偏差来得到输出。卷积层的模型参数包括了卷积核和标量偏差。在训练模型的时候，通常我们先对卷积核随机初始化，然后不断迭代卷积核和偏差。

下面基于 `corr2d` 函数来实现一个自定义的二维卷积层。在构造函数 `__init__` 里我们声明 `weight` 和 `bias` 这两个模型参数。前向计算函数 `forward` 则是直接调用 `corr2d` 函数再加上偏差。

In [3]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

卷积窗口形状为 $p×q$ 的卷积层称为 $p×q$ 卷积层。同样，$p×q$ 卷积或 $p×q$ 卷积核说明卷积核的高和宽分别为 $p$ 和 $q$。

In [4]:
layer = Conv2D(K.shape)
layer(X)

tensor([[ 2.9415,  6.7972],
        [14.5084, 18.3641]], grad_fn=<AddBackward0>)

## 图像中物体边缘检测

下面我们来看一个卷积层的简单应用：检测图像中物体的边缘，即找到像素变化的位置。首先我们构造一张 $6×8$ 的图像（即高和宽分别为 6 像素和 8 像素的图像）。它中间 4 列为黑（0），其余为白（1）。

In [5]:
X = torch.ones(6, 8)
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

然后我们构造一个高和宽分别为 1 和 2 的卷积核 `K`。当它与输入做互相关运算时，如果横向相邻元素相同，输出为 0；否则输出为非 0。

In [6]:
K = torch.tensor([[1, -1]])

下面将输入 `X` 和我们设计的卷积核 `K` 做互相关运算。可以看出，我们将从白到黑的边缘和从黑到白的边缘分别检测成了 1 和 -1。其余部分的输出全是 0。

In [7]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

由此，我们可以看出，卷积层可通过重复使用卷积核有效地表征局部空间。

现在我们将输入的二维图像转置，再进行如上的互相关运算。 其输出如下，之前检测到的垂直边缘消失了。 不出所料，这个卷积核 `K` 只可以检测垂直边缘，无法检测水平边缘。

In [8]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 通过数据学习核数组

如果我们只需寻找黑白边缘，那么以上 $[1, -1]$ 的边缘检测器足以。然而，当有了更复杂数值的卷积核，或者连续的卷积层时，我们不可能手动设计过滤器。那么我们是否可以学习由 `X` 生成 `Y` 的卷积核呢？

现在让我们看看是否可以通过仅查看“输入-输出”对来了解由 `X` 生成 `Y` 的卷积核。 我们先构造一个卷积层，并将其卷积核初始化为随机张量。接下来，在每次迭代中，我们比较 `Y` 与卷积层输出的平方误差，然后计算梯度来更新卷积核。为了简单起见，我们在此使用内置的二维卷积层。


In [9]:
# 构造一个核数组形状是(1, 2)的二维卷积层
conv2d = Conv2D(kernel_size=(1, 2))

step = 20
lr = 0.01
for i in range(step):
    Y_hat = conv2d(X)
    l = ((Y_hat - Y) ** 2).sum()
    l.backward()

    # 梯度下降
    conv2d.weight.data -= lr * conv2d.weight.grad
    conv2d.bias.data -= lr * conv2d.bias.grad

    # 梯度清0
    conv2d.weight.grad.fill_(0)
    conv2d.bias.grad.fill_(0)
    if (i + 1) % 5 == 0:
        print('Step %d, loss %.3f' % (i + 1, l.item()))

Step 5, loss 0.079
Step 10, loss 0.021
Step 15, loss 0.006
Step 20, loss 0.002


可以看到，20 次迭代后误差已经降到了一个比较小的值。现在来看一下学习到的卷积核的参数。

In [10]:
print("weight: ", conv2d.weight.data)
print("bias: ", conv2d.bias.data)

weight:  tensor([[ 1.0097, -1.0105]])
bias:  tensor([0.0005])


细心的你一定会发现，我们学习到的卷积核权重非常接近我们之前定义的卷积核 `K`。


## 小结

- 二维卷积层的核心计算是二维互相关运算。最简单的形式是，对二维输入数据和卷积核执行互相关操作，然后添加一个偏置。

- 我们可以设计一个卷积核来检测图像的边缘。

- 我们可以从数据中学习卷积核的参数。

- 学习卷积核时，无论用严格卷积运算或互相关运算，卷积层的输出不会受太大影响。

- 当需要检测输入特征中更广区域时，我们可以构建一个更深的卷积网络。

---

# 填充和步幅

>(Restart your kernel here)

在前面的例子中，输入的高度和宽度都为 $3$，卷积核的高度和宽度都为 $2$，生成的输出表征的维数为 $2\times2$。 正如我们在 PPT 中所概括的那样，假设输入形状为 $n_h\times n_w$，卷积核形状为 $k_h\times k_w$，那么输出形状将是 $(n_h-k_h+1) \times (n_w-k_w+1)$。 因此，卷积的输出形状取决于输入形状和卷积核的形状。

还有什么因素会影响输出的大小呢？本节我们将介绍 **填充**（padding）和 **步幅** (stride)。假设以下情景：

- 有时，在应用了连续的卷积之后，我们最终得到的输出远小于输入大小。这是由于卷积核的宽度和高度通常大于 1  所导致的。比如，一个  $240\times240$  像素的图像，经过  $10$  层  $5\times5$  的卷积后，将减少到  $200\times200$  像素。如此一来，原始图像的边界丢失了许多有用信息。 而填充 是解决此问题最有效的方法。
- 有时，我们可能希望大幅降低图像的宽度和高度。例如，如果我们发现原始的输入分辨率十分冗余。 步幅则可以在这类情况下提供

## 填充

如上所述，在应用多层卷积时，我们常常丢失边缘像素。 由于我们通常使用小卷积核，因此对于任何单个卷积，我们可能只会丢失几个像素。 但随着我们应用许多连续卷积层，累积丢失的像素数就多了。 解决这个问题的简单方法即为填充（padding）：在输入图像的边界填充元素（通常填充元素是  $0$  ）。 例如，在图下中，我们将  $3\times3$ 输入填充到  $5\times5$ ，那么它的输出就增加为 $4\times4$。阴影部分是第一个输出元素以及用于输出计算的输入和核张量元素： $0\times0+0\times1+0\times2+0\times3=0$。

![](https://i.loli.net/2021/04/17/Oyzk4LQC8UMd1Jt.png)



通常，如果我们添加  $p_h$ 行填充（大约一半在顶部，一半在底部）和 $p_w$ 列填充（左侧大约一半，右侧半），则输出形状将为

$$
(n_h-k_h+p_h+1)\times(n_w-k_w+p_w+1)
$$

这意味着输出的高度和宽度将分别增加 $p_h$ 和 $p_w$。

在许多情况下，我们需要设置 $p_h=k_h-1$ 和 $p_w=k_w-1$，使输入和输出具有相同的高度和宽度。 这样可以在构建网络时更容易地预测每个图层的输出形状。假设 $k_h$ 是奇数，我们将在高度的两侧填充 $p_h/2$ 行。 如果 $k_h$ 是偶数，则一种可能性是在输入顶部填充 $\lceil p_h/2\rceil$ 行，在底部填充 $\lfloor p_h/2\rfloor$ 行。同理，我们填充宽度的两侧。

卷积神经网络中卷积核的高度和宽度通常为奇数，例如 1、3、5 或 7。 选择奇数的好处是，保持空间维度的同时，我们可以在顶部和底部填充相同数量的行，在左侧和右侧填充相同数量的列。

此外，使用奇数核和填充也提供了书写上的便利。对于任何二维张量 X，当满足： 

1. 内核的大小是奇数； 
2. 所有边的填充行数和列数相同； 
3. 输出与输入具有相同高度和宽度，则可以得出：输出 Y[i, j] 是通过以输入 X[i, j] 为中心，与卷积核进行互相关计算。

比如，在下面的例子中，我们创建一个高度和宽度为 3 的二维卷积层，并在所有侧边填充 1 个像素。给定高度和宽度为  8  的输入，则输出的高度和宽度也是 8。

In [11]:
import torch
from torch import nn

# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的（1，1）表示批量大小和通道数都是 1
    X = X.reshape((1, 1) + X.shape)   # torch.Size([1, 1, 8, 8])
    Y = conv2d(X)
    # 省略前两个维度：批量大小和通道
    return Y.reshape(Y.shape[2:])

# 请注意，这里每边都填充了 1 行或 1 列，因此总共添加了 2 行或 2 列
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))

comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

当卷积内核的高度和宽度不同时，我们可以填充不同的高度和宽度，使输出和输入具有相同的高度和宽度。在如下示例中，我们使用高度为  5 ，宽度为  3  的卷积核，高度和宽度两边的填充分别为  2  和  1 。

In [12]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

## 步幅

在计算互相关时，卷积窗口从输入张量的左上角开始，向下和向右滑动。 在前面的例子中，我们默认每次滑动一个元素。 但是，有时候为了高效计算或是缩减采样次数，卷积窗口可以跳过中间位置，每次滑动多个元素。

我们将每次滑动元素的数量称为 步幅 （stride）。到目前为止，我们只使用过高度或宽度为  1  的步幅，那么如何使用较大的步幅呢？ 下面图是垂直步幅为  3 ，水平步幅为  2  的二维互相关运算。 着色部分是输出元素以及用于输出计算的输入和内核张量元素： $0\times0+0\times1+1\times2+2\times3=8$ 、 $0\times0+6\times1+0\times2+0\times3=6$。

如何计算输出中第一列的第二个元素呢？如图所示，卷积窗口向下滑动三行、向右滑动两列。但是，当卷积窗口继续向右滑动两列时，没有输出，因为输入元素无法填充窗口（除非我们添加另一列填充）。

![](https://i.loli.net/2021/04/17/9kObTZjpWGXIN6u.png)

通常，当垂直步幅为 $s_h$、水平步幅为 $s_w$ 时，输出形状为

$$
\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor \times \lfloor(n_w-k_w+p_w+s_w)/s_w\rfloor.
$$

如果我们设置了 $p_h=k_h-1$ 和 $p_w=k_w-1$，则输出形状将简化为 $\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w-1)/s_w\rfloor$。 更进一步，如果输入的高度和宽度可以被垂直和水平步幅整除，则输出形状将为 $(n_h/s_h) \times (n_w/s_w)$。

下面，我们将高度和宽度的步幅设置为  2 ，从而将输入的高度和宽度减半。

In [13]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

接下来，看一个稍微复杂的例子。

In [14]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

为了简洁起见，当输入高度和宽度两侧的填充数量分别为 $p_h$ 和 $p_w$ 时，我们称之为填充 $(p_h, p_w)$。当 $p_h = p_w = p$ 时，填充是 $p$ 。同理，当高度和宽度上的步幅分别为 $s_h$ 和 $s_w$ 时，我们称之为步幅 $(s_h, s_w)$ 。当时的步幅为 $s_h = s_w = s$ 时，步幅为 $s$。默认情况下，填充为 0，步幅为 1。在实践中，我们很少使用不一致的步幅或填充，也就是说，我们通常有  $p_h = p_w$  和 $s_h = s_w$。

## 小结

- 填充可以增加输出的高度和宽度。这常用来使输出与输入具有相同的高和宽。

- 步幅可以减小输出的高和宽，例如输出的高和宽仅为输入的高和宽的  $1/n$ （  $n$  是一个大于  1  的整数）。

- 填充和步幅可用于有效地调整数据的维度。

--- 

# 多输入多输出通道

>(Restart your kernel here)

前面两节里我们用到的输入和输出都是二维数组，但真实数据的维度经常更高。例如，彩色图像在高和宽 2 个维度外还有 RGB（红、绿、蓝）3个颜色通道。假设彩色图像的高和宽分别是 $h$ 和 $w$（像素），那么它可以表示为一个 $3\times h\times w$ 的多维数组。我们将大小为3的这一维称为**通道**（channel）维。本节我们将介绍含多个输入通道或多个输出通道的卷积核。

## 多输入通道

当输入包含多个通道时，需要构造一个与输入数据具有相同输入通道数目的卷积核，以便与输入数据进行互相关运算。假设输入的通道数为 $c_i$，那么卷积核的输入通道数也需要为 $c_i$。如果卷积核的窗口形状是 $k_h\times k_w$，那么当 $c_i=1$ 时，我们可以把卷积核看作形状为 $k_h\times k_w$ 的二维张量。

然而，当 $c_i>1$ 时，我们卷积核的每个输入通道将包含形状为 $k_h\times k_w$ 的张量。将这些张量 $c_i$ 连结在一起可以得到形状为 $c_i\times k_h\times k_w$ 的卷积核。由于输入和卷积核都有 $c_i$ 个通道，我们可以对每个通道输入的二维张量和卷积核的二维张量进行互相关运算，再对通道求和（将 $c_i$ 的结果相加）得到二维张量。这是多通道输入和多输入通道卷积核之间进行二维互相关运算的结果。

在下图中，我们演示了一个具有两个输入通道的二维互相关运算的示例。阴影部分是第一个输出元素以及用于计算这个输出的输入和核张量元素： $(1\times1+2\times2+4\times3+5\times4)+(0\times0+1\times1+3\times2+4\times3)=56$ 。

![](https://i.loli.net/2021/04/17/WuneXCDUlzmvN4a.png)

为了加深理解，我们将多输入通道互相关运算实现一下。 简而言之，我们所做的就是对每个通道执行互相关操作，然后将结果相加。

In [1]:
import torch
import dl4wm

def corr2d_multi_in(X, K):
    # 先遍历 “X” 和 “K” 的第0个维度（通道维度），再把它们加在一起
    return sum(dl4wm.corr2d(x, k) for x, k in zip(X, K))

我们可以构造与上图中的值相对应的输入张量 `X` 和核张量 `K`，以验证互相关运算的输出。

In [2]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

## 多输出通道

到目前为止，不论有多少输入通道，我们还只有一个输出通道。然而，正如我们在 PPT 中所讨论的，每一层有多个输出通道是至关重要的。在最流行的神经网络架构中，随着神经网络层数的加深，我们常会增加输出通道的维数，通过减少空间分辨率以获得更大的通道深度。直观地说，我们可以将每个通道看作是对不同特征的响应。而现实可能更为复杂一些，因为每个通道不是独立学习的，而是为了共同使用而优化的。因此，多输出通道并不仅是学习多个单通道的检测器。

用 $c_i$ 和 $c_o$ 分别表示输入和输出通道的数目，并让 $k_h$ 和 $k_w$ 为卷积核的高度和宽度。为了获得多个通道的输出，我们可以为每个输出通道创建一个形状为 $c_i\times k_h\times k_w$ 的卷积核张量，这样卷积核的形状是 $c_o\times c_i\times k_h\times k_w$。在互相关运算中，每个输出通道先获取所有输入通道，再以对应该输出通道的卷积核计算出结果。

如下所示，我们实现一个计算多个通道的输出的互相关函数。



In [3]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

通过将核张量 `K` 与 `K+1` （ `K` 中每个元素加  1  ）和 `K+2` 连接起来，构造了一个具有 3 个输出通道的卷积核。

In [4]:
print(K.shape) # 原核张量 K 的形状
K

torch.Size([2, 2, 2])


tensor([[[0., 1.],
         [2., 3.]],

        [[1., 2.],
         [3., 4.]]])

In [5]:
K = torch.stack((K, K + 1, K + 2), dim=0)
K.shape

torch.Size([3, 2, 2, 2])

下面，我们对输入张量 `X` 与卷积核张量 `K` 执行互相关运算。现在的输出包含  3  个通道，第一个通道的结果与先前输入张量 `X` 和多输入单输出通道的结果一致。

In [7]:
corr2d_multi_in_out(X, K).shape

torch.Size([3, 2, 2])

## $1\times1$  卷积层

$1\times1$  卷积，即 $k_h = k_w = 1$，看起来似乎没有多大意义。毕竟，卷积的本质是有效提取相邻像素间的相关特征，而 $1\times1$ 卷积显然没有此作用。 尽管如此，$1\times1$ 仍然十分流行，时常包含在复杂深层网络的设计中。下面，让我们详细地解读一下它的实际作用。

因为使用了最小窗口，$1\times1$ 卷积失去了卷积层的特有能力——在高度和宽度维度上，识别相邻元素间相互作用的能力。 而 $1\times1$ 卷积的唯一计算发生在通道上。

下图展示了使用 $1\times1$ 卷积核与  3  个输入通道和  2  个输出通道的互相关计算。 这里输入和输出具有相同的高度和宽度，输出中的每个元素都是从输入图像中同一位置的元素的线性组合。 我们可以将 $1\times1$ 卷积层看作是在每个像素位置应用的全连接层，以 $c_i$ 个输入值转换为 $c_o$ 个输出值。 因为这仍然是一个卷积层，所以跨像素的权重是一致的。 同时，$1\times1$ 卷积层需要的权重维度为  $c_o\times c_i$ ，再额外加上一个偏置。

![](https://i.loli.net/2021/04/17/wTURXzFfsjiNOYq.png)

下面，我们使用全连接层实现 $1\times1$ 卷积。 请注意，我们需要对输入和输出的数据形状进行微调。

In [8]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape  # (3,3,3)
    c_o = K.shape[0]     # 2
    X = X.reshape((c_i, h * w)) # (3,3*3)
    K = K.reshape((c_o, c_i)) # (2,3)
    Y = torch.matmul(K, X)  # 全连接层中的矩阵乘法 # (2,9)
    return Y.reshape((c_o, h, w)) # (2,3,3)

当执行 $1\times1$ 卷积运算时，上述函数相当于先前实现的互相关函数 `corr2d_multi_in_out`。让我们用一些样本数据来验证这一点。

In [9]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

在之后的模型里我们将会看到 $1\times1$ 卷积层被当作保持高和宽维度形状不变的全连接层使用。于是，我们可以通过调整网络层之间的通道数来控制模型复杂度。

## 小结

- 多输入多输出通道可以用来扩展卷积层的模型。

- 当以每像素为基础应用时，$1\times1$ 卷积层相当于全连接层。

- $1\times1$ 卷积层通常用于调整网络层的通道数量和控制模型复杂性。

---

# 池化层

>(Restart your kernel here)

通常当我们处理图像时，我们希望逐渐降低隐藏表示的空间分辨率，聚集信息，这样的随着我们在神经网络中层叠的上升，每个神经元对其敏感的感受野（输入）就越大。

而我们的机器学习任务通常会跟全局图像的问题有关（例如，“图像是否包含一只猫呢？”）， 所以我们最后一层的神经元应该对整个输入的全局敏感。通过逐渐聚合信息，生成越来越粗糙的映射，最终实现学习全局表示的目标，同时将卷积图层的所有优势保留在中间层。

此外，当检测较底层的特征时（例如上面「图像中物体边缘检测」节中所讨论的边缘），我们通常希望这些特征保持某种程度上的平移不变性。例如，如果我们拍摄黑白之间轮廓清晰的图像 X，并将整个图像向右移动一个像素，即 Z[i, j] = X[i, j + 1]，则新图像 Z 的输出可能大不相同。而在现实中，随着拍摄角度的移动，任何物体几乎不可能发生在同一像素上。即使用三脚架拍摄一个静止的物体，由于快门的移动而引起的相机振动，可能会使所有物体左右移动一个像素（除了高端相机配备了特殊功能来解决这个问题）。

本节将介绍 **池化**（pooling）层，它具有双重目的：

- 降低卷积层对位置的敏感性
- 同时降低对空间降采样表示的敏感性。


## 最大池化层和平均池化层

与卷积层类似，池化层运算符由一个固定形状的窗口组成，该窗口根据其步幅大小在输入的所有区域上滑动，为固定形状窗口（有时称为 池化窗口）遍历的每个位置计算一个输出。 然而，不同于卷积层中的输入与卷积核之间的互相关计算，池化层不包含参数。 相反，池运算符是确定性的，我们通常计算池化窗口中所有元素的最大值或平均值。这些操作分别称为 **最大池化层** （maximum pooling）和 **平均池化层** （average pooling）。

在这两种情况下，与互相关运算符一样，池化窗口从输入张量的左上角开始，从左到右、从上到下的在输入张量内滑动。在池化窗口到达的每个位置，它计算该窗口中输入子张量的最大值或平均值，具体取决于是使用了最大池化层还是平均池化层。

![](https://i.loli.net/2021/04/17/ROk5GSYXwJm3HsC.png)

上图中的输出张量的高度为  2 ，宽度为  2 。这四个元素为每个池化窗口中的最大值：

$$
\begin{split}\max(0, 1, 3, 4)=4,\\
\max(1, 2, 4, 5)=5,\\
\max(3, 4, 6, 7)=7,\\
\max(4, 5, 7, 8)=8.\\\end{split}
$$

池化窗口形状为 $p\times q$  的池化层称为 $p\times q$ 池化层，池化操作称为 $p\times q$ 池化。

回到本节开头提到的对象边缘检测示例，现在我们将使用卷积层的输出作为 $2\times 2$ 最大池化的输入。 设置卷积层输入为 X，池化层输出为 Y。 无论 X[i, j] 和 X[i, j + 1] 的值是否不同，或 X[i, j + 1] 和 X[i, j + 2] 的值是否不同，池化层始终输出 Y[i, j] = 1。 也就是说，使用 $2\times 2$ 最大池化层，即使在高度或宽度上移动一个元素，卷积层仍然可以识别到模式。

在下面的代码中的 `pool2d` 函数，实现了池化层的正向传播。 此功能类似于 6.2节 中的 `corr2d` 函数。 然而，这里我们没有卷积核，输出为输入中每个区域的最大值或平均值。

In [11]:
import torch
from torch import nn

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i:i + p_h, j:j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i:i + p_h, j:j + p_w].mean()
    return Y

我们可以构建上图中的输入张量 `X`，验证二维最大池化层的输出。

In [12]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, pool_size=(2, 2), mode='max')

tensor([[4., 5.],
        [7., 8.]])

此外，我们还可以验证平均池化层。

In [13]:
pool2d(X, pool_size=(2, 2), mode='avg')

tensor([[2., 3.],
        [5., 6.]])

## 填充和步幅

与卷积层一样，池化层也可以改变输出形状。和以前一样，我们可以通过填充和步幅以获得所需的输出形状。 下面我们将通过 `nn` 模块里的二维最大池化层 `MaxPool2d` 来演示池化层填充和步幅的工作机制。我们首先构造了一个输入张量 `X`，它有四个维度，形状为(1, 1, 4, 4)，前两个维度分别是批量和通道，其中样本数和通道数都是 1。

In [14]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

默认情况下，深度学习框架中的步幅与池化窗口的大小相同。 因此，如果我们使用形状为 (3, 3) 的池化窗口，那么默认情况下，我们得到的步幅形状为 (3, 3)。

In [15]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

填充和步幅可以手动设定。

In [16]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

当然，我们可以设定一个任意大小的非正方形的池化窗口，并分别设定填充和步幅的高度和宽度。

In [17]:
pool2d = nn.MaxPool2d((2, 3), padding=(1, 1), stride=(2, 3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

## 多通道

在处理多通道输入数据时，池化层在每个输入通道上单独运算，而不是像卷积层一样在通道上对输入进行汇总。 这意味着池化层的输出通道数与输入通道数相同。 下面，我们将在通道维度上连结张量 `X` 和 `X + 1`，以构建具有 2 个通道的输入。

In [18]:
X = torch.cat((X, X + 1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [19]:
X.shape

torch.Size([1, 2, 4, 4])

如下所示，池化后输出通道的数量仍然是 2。

In [20]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

## 小结

- 对于给定输入元素，最大池化层会输出该窗口内的最大值，平均池化层会输出该窗口内的平均值。

- 池化层的主要优点之一是减轻卷积层对位置的过度敏感。

- 我们可以指定池化层的填充和步幅。

- 使用最大池化层以及大于 1 的步幅，可减少空间维度（如高度和宽度）。

- 池化层的输出通道数与输入通道数相同。

---

# 卷积神经网络（LeNet）

>(Restart your kernel here)

通过之前几节，我们学习了构建一个完整卷积神经网络的所需组件。 回想一下，之前我们将 「softmax 回归模型」和「多层感知机模型」应用于 Fashion-MNIST 数据集中的服装图片上。 为了能够应用 softmax 回归和多层感知机，我们首先将每个大小为  $28\times28$ 的图像展平为一个 784 固定长度的一维向量，然后用全连接层对其进行处理。 而现在，我们已经掌握了卷积层的处理方法，我们可以在图像中保留空间结构。 同时，用卷积层代替全连接层的另一个好处是：更简洁的模型所需的参数更少。

在本节中，我们将介绍 **LeNet**，它是最早发布的卷积神经网络之一，因其在计算机视觉任务中的高效性能而受到广泛关注。 这个模型是由 AT&T 贝尔实验室的研究员 Yann LeCun 在1989年提出的（并以其命名），目的是识别图像 [^LeCun1998] 中的手写数字。 当时，Yann LeCun 发表了第一篇通过反向传播成功训练卷积神经网络的研究，这项工作代表了十多年来神经网络研究开发的成果。

当时， LeNet 取得了与支持向量机（support vector machines）性能相媲美的成果，成为监督学习的主流方法。 LeNet 被广泛用于自动取款机（ATM）机中，帮助识别处理支票的数字。 时至今日，一些自动取款机仍在运行 Yann LeCun 和他的同事 Leon Bottou 在上世纪90年代写的代码呢！

[^LeCun1998]: LeCun, Y., Bottou, L., Bengio, Y., Haffner, P., & others. (1998). Gradient-based learning applied to document recognition. Proceedings of the IEEE, 86(11), 2278–2324.

## LeNet

总体来看，LeNet (LeNet-5) 由两个部分组成：

- 卷积编码器：由两个卷积层组成;

- 全连接层密集块：由三个全连接层组成。

该结构在下图中所展示。

![](https://i.loli.net/2021/04/17/tQBqM3nKFIvaujm.png)

每个卷积块中的基本单元是一个卷积层、一个 sigmoid 激活函数和平均池化层。请注意，虽然 ReLU 和最大池化层更有效，但它们在20世纪90年代还没有出现。每个卷积层使用  $5\times5$  卷积核，这些层将输入映射到多个二维特征输出，通常同时增加通道的数量。第一卷积层有 6 个输出通道，而第二个卷积层有 16 个输出通道。每个 $2\times2$  池操作（步骤2）通过空间下采样将维数减少 4 倍。卷积的输出形状由批量大小、通道数、高度、宽度决定。

为了将卷积块的输出传递给稠密块，我们必须在小批量中展平每个样本。换言之，我们将这个四维输入转换成全连接层所期望的二维输入。这里的二维表示的第一个维度索引小批量中的样本，第二个维度给出每个样本的平面向量表示。LeNet 的稠密块有三个全连接层，分别有 120、84 和 10 个输出。因为我们仍在执行分类，所以输出层的 10 维对应于最后输出结果的数量。

通过下面的 LeNet 代码，你会相信用深度学习框架实现此类模型非常简单。我们只需要实例化一个 `Sequential` 块并将需要的层连接在一起。

In [21]:
import torch
from torch import nn
import dl4wm
import time

class Reshape(torch.nn.Module):
    def forward(self, x):
        return x.view(-1, 1, 28, 28)

net = torch.nn.Sequential(Reshape(), nn.Conv2d(1, 6, kernel_size=5,
                                               padding=2), nn.Sigmoid(),
                          nn.AvgPool2d(kernel_size=2, stride=2),
                          nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
                          nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
                          nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
                          nn.Linear(120, 84), nn.Sigmoid(), nn.Linear(84, 10))

我们对原始模型做了一点小改动，去掉了最后一层的高斯激活。除此之外，这个网络与最初的 `LeNet-5` 一致。

下面，我们将一个大小为  $28\times28$  的单通道（黑白）图像通过 LeNet。 通过在每一层打印输出的形状，我们可以检查模型，以确保其操作与我们期望的上图一致。

![](https://i.loli.net/2021/04/17/wSbi4QhG7C9N3ol.png)

In [22]:
X = torch.rand(size=(1, 1, 28, 28), dtype=torch.float32)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape: \t', X.shape)

Reshape output shape: 	 torch.Size([1, 1, 28, 28])
Conv2d output shape: 	 torch.Size([1, 6, 28, 28])
Sigmoid output shape: 	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape: 	 torch.Size([1, 6, 14, 14])
Conv2d output shape: 	 torch.Size([1, 16, 10, 10])
Sigmoid output shape: 	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape: 	 torch.Size([1, 16, 5, 5])
Flatten output shape: 	 torch.Size([1, 400])
Linear output shape: 	 torch.Size([1, 120])
Sigmoid output shape: 	 torch.Size([1, 120])
Linear output shape: 	 torch.Size([1, 84])
Sigmoid output shape: 	 torch.Size([1, 84])
Linear output shape: 	 torch.Size([1, 10])


请注意，在整个卷积块中，与上一层相比，每一层特征的高度和宽度都减小了。 第一个卷积层使用 2 个像素的填充，来补偿  5×5  卷积核导致的特征减少。 相反，第二个卷积层没有填充，因此高度和宽度都减少了 4 个像素。 随着层叠的上升，通道的数量从输入时的 1 个，增加到第一个卷积层之后的 6 个，再到第二个卷积层之后的 16 个。 同时，每个池化层的高度和宽度都减半。最后，每个全连接层减少维数，最终输出一个维数与结果分类数相匹配的输出。

## 模型训练

现在我们已经实现了 LeNet ，让我们看看这个模型在 Fashion-MNIST 数据集上的表现。

In [3]:
batch_size = 256
train_iter, test_iter = dl4wm.load_data_fashion_mnist(batch_size=batch_size)

虽然卷积神经网络的参数较少，但与深度的多层感知机相比，它们的计算成本仍然很高，因为每个参数都参与更多的乘法。 如果你有机会使用GPU，可以用它加快训练。

In [4]:
# 本函数已保存在 dl4wm 包中方便以后使用。
def try_gpu():
    """If GPU is available, return torch.device as cuda:0; else return torch.device as cpu."""
    return torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 本函数已保存在 dl4wm 包中方便以后使用。该函数将被逐步改进。
def evaluate_accuracy_gpu(data_iter, net,device=torch.device('cpu')):
    """使用GPU计算模型在数据集上的精度。"""
    acc_sum,n = torch.tensor([0],dtype=torch.float32,device=device),0
    for X,y in data_iter:
        # If device is the GPU, copy the data to the GPU.
        X,y = X.to(device),y.to(device)
        net.eval()
        with torch.no_grad():
            y = y.long()
            acc_sum += torch.sum((torch.argmax(net(X), dim=1) == y))
            n += y.shape[0]
    return acc_sum.item()/n

为了使用 GPU，我们还需要一点小改动。 与「softmax 回归的从零开始实现」中定义的 `train_cpu` 不同，在进行正向和反向传播之前，我们需要将每一小批量数据移动到我们指定的设备（例如 GPU）上。

如下所示，训练函数 `train_gpu` 也类似于「softmax 回归的从零开始实现」节中定义的 `train_cpu`。 由于我们将实现多层神经网络，因此我们将主要使用高级 API。 以下训练函数假定从高级 API 创建的模型作为输入，并进行相应的优化。 我们使用在「数值稳定性和模型初始化」节 中介绍的 Xavier 随机初始化模型参数。 与全连接层一样，我们使用交叉熵损失函数和小批量随机梯度下降。

In [5]:
# 本函数已保存在 dl4wm 包中方便以后使用
def train_gpu(net, train_iter, test_iter,criterion,optimizer,num_epochs, batch_size, device,lr=None):
    """Train and evaluate a model with CPU or GPU."""
    print('training on', device)
    net.to(device)

    for epoch in range(num_epochs):
        train_l_sum = torch.tensor([0.0],dtype=torch.float32,device=device)
        train_acc_sum = torch.tensor([0.0],dtype=torch.float32,device=device)
        n, start = 0, time.time()
        for X, y in train_iter:
            net.train()
            
            optimizer.zero_grad()
            X,y = X.to(device),y.to(device) 
            y_hat = net(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                y = y.long()
                train_l_sum += loss.float()
                train_acc_sum += (torch.sum((torch.argmax(y_hat, dim=1) == y))).float()
                n += y.shape[0]
        test_acc = evaluate_accuracy_gpu(test_iter, net,device)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum/n, train_acc_sum/n, test_acc,
                 time.time() - start))

现在，我们训练和评估 LeNet-5 模型。

学习率采用 0.001，训练算法使用 Adam 算法，损失函数使用交叉熵损失函数。

In [7]:
lr, num_epochs = 0.001, 5

def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)

net.apply(init_weights)

optimizer = torch.optim.Adam(net.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
train_gpu(net, train_iter, test_iter, criterion, optimizer, num_epochs, batch_size, try_gpu(), lr)

training on cpu
epoch 1, loss 0.0073, train acc 0.318, test acc 0.602, time 34.8 sec
epoch 2, loss 0.0034, train acc 0.684, test acc 0.706, time 35.8 sec
epoch 3, loss 0.0028, train acc 0.733, test acc 0.743, time 33.0 sec
epoch 4, loss 0.0025, train acc 0.755, test acc 0.757, time 33.9 sec
epoch 5, loss 0.0023, train acc 0.772, test acc 0.768, time 43.1 sec


## 小结

- 卷积神经网络（CNN）是一类使用卷积层的网络。

- 在卷积神经网络中，我们组合使用卷积层、非线性激活函数和池化层。

- 为了构造高性能的卷积神经网络，我们通常对卷积层进行排列，逐渐降低其表示的空间分辨率，同时增加通道数。

- 在传统的卷积神经网络中，卷积块编码得到的表征在输出之前需由一个或多个全连接层进行处理。

- LeNet 是最早发布的卷积神经网络之一。